In [1]:
from glob import glob
import os

import cv2
import numpy as np
import torch
from PIL import Image
from pdf2image import convert_from_path
from tqdm import tqdm
from scoring import *
from border_and_title import *

good = list(glob("../../data/original/Good/*.pdf"))
ugly = list(glob("../../data/original/Ugly/*.pdf"))

len(good), len(ugly)

/home/sadid/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


(16, 25)

In [2]:
def process(files, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (0, 255, 0)
    thickness = 2

    progress = tqdm(total=len(files))

    for f in files:
        pages = convert_from_path(f, dpi=300)
        img = np.array(pages[0])
        img = im_resize(img)
        img_comp = img.copy()
        im_h, im_w, _ = img.shape

        sorted_indices, contours, hierarchy_area = detect_objects(img)
        title_contours, line_90, data = detect_probable_title_sections(img, return_states=True)
        ocr_result = data["ocr_result"]
        mask = data["mask"]
        inner_border_lines = data["inner_border_lines"]
        words = process_text(ocr_result, im_h, im_w)

        border_1, border_2 = detect_borders(contours, sorted_indices, words)
        (x1, y1), (x2, y2) = border_1
        cv2.rectangle(img_comp, (x1, y1), (x2, y2), color, thickness)
        (x1, y1), (x2, y2) = border_2
        cv2.rectangle(img_comp, (x1, y1), (x2, y2), color, thickness)

        for line in inner_border_lines:
            x1, y1, x2, y2 = line
            cv2.line(img_comp, (x1, y1), (x2, y2), color, thickness)

        contours = detect_text_tables(img, words, mask)
        title_contours = title_contours + contours

        # bb = []
        # for cnt in contours:
        #     x, y, w, h = cv2.boundingRect(cnt)
        #     aspect_ratio = w / float(h)
        #     area = w * h
        #
        #     if area > 7000 and 1.3 < aspect_ratio < 10:
        #         if y > int((im_h * 70) / 100) and y+h > int((im_h * 93) / 100):
        #             cv2.rectangle(img_comp, (x, y), (x + w, y + h), (255, 0, 0), 10)
        #             bb.append((x, y, x+w, y+h))
        #         elif y > int((im_h * 80) / 100) and y+h > int((im_h * 90) / 100):
        #             cv2.rectangle(img_comp, (x, y), (x + w, y + h), (255, 0, 0), 10)
        #             bb.append((x, y, x+w, y+h))

        bb = []
        for cnt in title_contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x1 = x + w
            y1 = y + h
            aspect_ratio = w / float(h)
            area = w * h

            if area > 7000 and 1.3 < aspect_ratio < 10:
                ratio = table_content_score(((x, y), (x1, y1)), words)

                if ratio <= 1:
                    if y > int((im_h * 70) / 100) and y+h > int((im_h * 93) / 100):
                        # cv2.rectangle(img_comp, (x, y), (x1, y1), (100, 0, 0), 3)
                        bb.append((x, y, x1, y1))
                        # cv2.putText(img_comp, f"{ratio:.2f}", (x, y-10), font, fontScale, color, thickness)

                    elif y > int((im_h * 80) / 100) and y1 > int((im_h * 90) / 100):
                        # cv2.rectangle(img_comp, (x, y), (x1, y1), (180, 0, 0), 3)
                        bb.append((x, y, x1, y+h))
                        # cv2.putText(img_comp, f"{ratio:.2f}", (x, y-10), font, fontScale, color, thickness)
                # else:
                #     cv2.rectangle(img_comp, (x, y), (x1, y1), (0, 255, 0), 3)
                #     cv2.putText(img_comp, f"{ratio:.2f}", (x, y-10), font, fontScale, (0, 255, 0), thickness)
        try:
            bb = np.array(bb)

            if line_90 is not None:
                x1, y1, x2, y2 = line_90
                x1 = border_2[0][0]
                x2 = border_2[1][0]
                # cv2.line(img_comp, (x1, y1), (x2, y2), color, 5)
            else:
                x2 = bb[:, 2].max()
                x1 = bb[:, 0].min()
                y1 = bb[:, 1].min()
                y2 = border_2[1][1]

                if abs(border_2[1][0] - bb[:, 2].max()) < 50:
                    x2 = border_2[1][0]

                if abs(border_2[0][0] - bb[:, 0].min()) < 50:
                    x1 = border_2[0][0]

            cv2.rectangle(img_comp, (x1, y1), (x2, y2), color, thickness)
        except IndexError:
            print(f)
            print(bb)

        cv2.imwrite(f"{output_dir}/{os.path.basename(f)}.jpg", img_comp)
        progress.update(1)

In [3]:
process(ugly, "../../data/text-tables-detection/ugly")
process(good, "../../data/text-tables-detection/good")

 28%|██▊       | 7/25 [00:17<00:45,  2.53s/it]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 96%|█████████▌| 24/25 [00:59<00:02,  2.64s/it]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 38%|███▊      | 6/16 [00:17<00:26,  2.68s/it]

../../data/original/Good/Apple.pdf
[]


100%|██████████| 16/16 [00:52<00:00,  3.29s/it]
